In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199769 sha256=55ef9a7e3aad09f4f0dbd95b989210b140cc00c9342ed2c274b053fbaae74e02
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


## **Import the file and read it into a dataframe**

Import:

In [3]:
from google.colab import files

uploaded = files.upload()
# upload workshop file here

Use pandas for data reading and take aquick look:

In [2]:
import pandas as pd

df = pd.read_csv('workshop_set.csv')

FileNotFoundError: ignored

In [ ]:
df.head()

,Yelp URL,Rating,Date,Review Text
0,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,1/22/2022,All I can say is they have very good ice cream...
1,https://www.yelp.com/biz/sidney-dairy-barn-sidney,4,6/26/2022,Nice little local place for ice cream.My favor...
2,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,8/7/2021,A delicious treat on a hot day! Staff was very...
3,https://www.yelp.com/biz/sidney-dairy-barn-sidney,4,7/28/2016,This was great service and a fun crew! I got t...
4,https://www.yelp.com/biz/sidney-dairy-barn-sidney,5,6/23/2015,This is one of my favorite places to get ice c...


In [ ]:
# splitting data to workshop and test --> already done
# from sklearn.model_selection import train_test_split
# workshop_set, final_test_set = train_test_split(df2, test_size = 0.1, random_state =42)
# workshop_set.to_csv('workshop_set.csv', index=False)
# files.download('workshop_set.csv')

In [ ]:
import scattertext
import spacy
nlp = spacy.load('en_core_web_sm')

corpus = (scattertext.CorpusFromPandas(df,
                                           category_col='Rating',
                                           text_col='Review Text',
                                           nlp=nlp)
              .build()
              .remove_terms(nlp.Defaults.stop_words, ignore_absences=True)
              )

# html = scattertext.produce_scattertext_explorer(
#                    corpus,
#                    category='Low Rating',
#                    category_name='Low Rating',
#                    not_category_name='High Rating',
#                    width_in_pixels=1000,
#                    metadata=df['name'])


ModuleNotFoundError: ignored

## **Split data into train, test, validation**

In [ ]:
from sklearn.model_selection import train_test_split

train_df, test_and_val_df = train_test_split(workshop_set, test_size=0.3, random_state=42)
test_df, val_df = train_test_split(test_and_val_df, test_size=0.5, random_state=42)


In [ ]:
display(train_df.describe())
display(test_df.describe())
display(val_df.describe())

## **apply model**
1. put data in right format
2. train

In [ ]:
## put data in right format for fasttext
def map_adjusted_rating(rating):
    if rating in [1, 2]:
        return '__bad__'
    elif rating == 3:
        return '__neutral__'
    elif rating in [4, 5]:
        return '__good__'

# adjust train data
train_df['adjusted_rating'] = train_df['Rating'].apply(lambda x:(map_adjusted_rating(x)))
train_df['fasttext_data'] = train_df['adjusted_rating'] + ' ' + train_df['Review Text']
train_df['fasttext_data'].to_csv('train_fasttext.txt',header=None, index=None, quoting=None)
df_fasttext_training = pd.read_csv('train_fasttext.txt',sep='\t',header=None, names=['text'])
files.download('train_fasttext.txt')



In [ ]:
# adjust test data
test_df['adjusted_rating'] = test_df['Rating'].apply(lambda x:(map_adjusted_rating(x)))
test_df['fasttext_data'] = test_df['adjusted_rating'] + ' ' + test_df['Review Text']
test_df['fasttext_data'].to_csv('test_fasttext.txt',header=None, index=None, quoting=None)
df_fasttext_testing = pd.read_csv('test_fasttext.txt',sep='\t',header=None, names=['text'])
files.download('test_fasttext.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

training:

In [ ]:
import fasttext
# model_classification = fasttext.train_supervised(input='train_fasttext.txt', autotuneValidationFile='test_fasttext.txt',autotuneDuration=120, loss='ova')


## **Cleaning**
1. case based cleaning
2. remove punctuation
3. remove stop words (words that add nothing)
4. remove unimportant common words
5. inspect good and bad and remove more words


###1. case based cleaning.
make each word in the text lower case.

In [ ]:
df = df2
df["lowercase"] = df["Review Text"].apply(lambda x: " ".join(word.lower() for word in x.split()))

df.loc[:,['lowercase', 'Review Text']].head()

,lowercase,Review Text
0,all i can say is they have very good ice cream...,All I can say is they have very good ice cream...
1,nice little local place for ice cream.my favor...,Nice little local place for ice cream.My favor...
2,a delicious treat on a hot day! staff was very...,A delicious treat on a hot day! Staff was very...
3,this was great service and a fun crew! i got t...,This was great service and a fun crew! I got t...
4,this is one of my favorite places to get ice c...,This is one of my favorite places to get ice c...


###2. punctuation and regular expression
 can remove unwanted elements such as punctuation using a regular expression.

 handy regular expressions:
 - for punctation [^\w\s]
 - for numbers (\d+)

In [ ]:

df["punctuation"] = df["lowercase"].str.replace('[^\w\s]','')
df.loc[:,['punctuation', 'lowercase', 'Review Text']].head()

<ipython-input-78-27612036eb4a>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["punctuation"] = df["lowercase"].str.replace('[^\w\s]','')


,punctuation,lowercase,Review Text
0,all i can say is they have very good ice cream...,all i can say is they have very good ice cream...,All I can say is they have very good ice cream...
1,nice little local place for ice creammy favori...,nice little local place for ice cream.my favor...,Nice little local place for ice cream.My favor...
2,a delicious treat on a hot day staff was very ...,a delicious treat on a hot day! staff was very...,A delicious treat on a hot day! Staff was very...
3,this was great service and a fun crew i got th...,this was great service and a fun crew! i got t...,This was great service and a fun crew! I got t...
4,this is one of my favorite places to get ice c...,this is one of my favorite places to get ice c...,This is one of my favorite places to get ice c...


###3. removing stop words
stop words are words that add no significant meaning. Can do this using available dataset, e.g. nltk.

In [ ]:
## 3
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stopwords = stopwords.words('english')
stopwords

In [ ]:
df['stopwords'] = df['punctuation'].apply(lambda x: " ".join(word for word in x.split() if word not in stopwords))
df.loc[:,['stopwords','punctuation', 'lowercase', 'Review Text']].head()

###4. removing words that are common but not valuable through inspection

inspect the whole dataset or inspect only certain ratings

In [ ]:
## 4
pd.Series(" ".join(df['stopwords']).split()).value_counts()[:50]

In [ ]:
## 4
other_stop_words = [ "also", "would", "got", "first", "one", "cream", "ice"]
df['cleaned'] = df['stopwords'].apply(lambda x: " ".join(word for word in x.split() if word not in other_stop_words))
df.loc[:,['cleaned', 'stopwords','punctuation', 'lowercase', 'Review Text']].head()

inspect only certain ratings:

In [ ]:
## 5
bad_df = df.apply(lambda row: row['cleaned'] if row['Rating']==1 else "", axis=1)
bad_series = pd.Series(" ".join(bad_df).split()).value_counts()[:25]
display(bad_series)

good_df = df.apply(lambda row: row['cleaned'] if row['Rating']==5 else "", axis=1)
good_series = pd.Series(" ".join(good_df).split()).value_counts()[:25]
display(good_series)



## Apply model again on cleaned dataset and see difference in results